In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import random as rd
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as tud
# from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import CustomDataset as cd
import Net

In [10]:
def train(model, device, train_loader, optimizer, epoch, batch_size = 50):
    model.train()
    #loop run over data output by loader to train model
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data) #applies model to data
        # print(target.shape)
        target = target.squeeze(1) #removes dimension from target
        # print(target.shape)
        # print(output)
        # print(target.shape)
        # loss = nn.CrossEntropyLoss()(output, target[:,1].long()) #calculates cross entropy loss
    
        loss = nn.CrossEntropyLoss()(output, target.long())
        # print(loss)
        model.zero_grad()
        loss.backward()
        optimizer.step() #new step in optimisation of loss
        #printing output of each batch for loss observation while model is training
        if batch_idx % batch_size == 0:
            print(
                'Train Epoch: {} [{}/{} ({}%)]\tLoss: {}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset), \
                    round(100 * batch_idx / len(train_loader),0), round(loss.item(),5)
                )
            )

In [13]:
def test(model, device, test_loader):
    model.eval()
    #initialisting parameters and empty lists
    test_loss = 0
    correct = 0
    probs_pred = []
    labels_actual = []
    labels_pred = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            target=target.squeeze(1)
            # print(output.shape)
            # test_loss += nn.CrossEntropyLoss()(output, target[:,1].long()).item()
            # print(output)
            test_loss += nn.CrossEntropyLoss()(output, target.long()).item()

            # pred = output.max(1, keepdim=True)[1] #finds index of maximum in output vector as described below

            pred = output.max(1, keepdim=False)[1]#torch.round(output)

            # print(output)
            # pred_onehot = F.one_hot(pred).squeeze(1) #converts into one-hot predicted label 
            # print(pred_onehot)
            # print(test_loss)
            # pred_bool = torch.eq(pred_onehot,target) #finds values where predicted equals target
            pred_bool = torch.eq(pred,target)
            # print(pred_bool)
            # correct += int((pred_bool.sum().item())/2) #sums number of correct values
            correct += pred_bool.sum().item()
            #adds predicted labels, targets and probabilities to lists for later use
            labels_actual.append(target.cpu().numpy())
            probs_pred.append(output.cpu().numpy())
            # labels_pred.append(pred_onehot.cpu().numpy())

    #printing accuracy and loss after each epoch for analysis
    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {}, Accuracy: {}/{} ({}%)\n'.format(
              round(test_loss,4), correct, len(test_loader.dataset), \
            round(100 * correct / len(test_loader.dataset),0)
            )
          )
  
    return(labels_actual,probs_pred,labels_pred)

In [12]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

"""
For a learning algorithm, it will be necesarry to have an input (X data) containing a number of $97\times97$ matrices for each energy. 
The output will be a label, either Carbon or Flourine. The model will compare its output with the actual labels and adjust accordingly to minimise loss. 

In order to quantify each label, the common 'one-hot encoding' method will be used. 
C elements will have the label `[1, 0]` and F will be `[0, 1]`. 
Tuples of this form will be the ground truth labels and the model output. 
I will have to ensure that the model outputs an array of shape `[2]`.
"""

# image data, one-hot elem labels, energy
# im_dat, el_labs, en_labs = [],[],[]
im_dat, el_labs = [],[]

energies = np.arange(50,196,5)

for i in range(len(energies)): # run over all energies
    E = energies[i]

    filename = 'data/C_'+str(E)+'keV.npy'
    print("Loading {}...".format(filename))

    C_dat = np.load(filename)[:,:,:]
    # Can scale by mean and std:
    C_dat = (C_dat-np.nanmean(C_dat,axis=0))/np.nanstd(C_dat,axis=0)
    # C_dat = (C_dat-np.nanmean(C_dat,axis=(1,2)).reshape(-1,1,1))/np.nanstd(C_dat,axis=(1,2)).reshape(-1,1,1)
    C_dat[C_dat!=C_dat] = 0

    for n in range(np.shape(C_dat)[0]):
        im_dat.append(C_dat[n])
        # el_labs.append([1,0])
        el_labs.append(i)

        # en_labs.append(i)

    filename = 'data/F_'+str(E)+'keV.npy'
    print("Loading {}...".format(filename))

    F_dat = np.load(filename)[:,:,:]
    F_dat = (F_dat-np.nanmean(F_dat,axis=0))/np.nanstd(F_dat,axis=0)
    # F_dat = (F_dat-np.nanmean(F_dat,axis=(1,2)).reshape(-1,1,1))/np.nanstd(F_dat,axis=(1,2)).reshape(-1,1,1)
    F_dat[F_dat!=F_dat] = 0

    for n in range(np.shape(F_dat)[0]):
        im_dat.append(F_dat[n])
        # el_labs.append([0,1])
        el_labs.append(i+len(energies)) # offset fluorines
        # en_labs.append(i)
        
# multiclass classifying [carbon energies... fluorine energies]
# Sum up probabilities of C's and F's to determine whether carbon or fluorine
# Can then find the peak of the distribution to extract the energy?
# Or just take most probable energy?        

# C_dat = np.load('C_130keV.npy')

# for n in range(np.shape(C_dat)[0]):
#   im_dat.append(C_dat[n])
#   el_labs.append([1,0])
#   en_labs.append(130)
  
# F_dat = np.load('F_170keV.npy')

# for n in range(np.shape(F_dat)[0]):
#   im_dat.append(F_dat[n])
#   el_labs.append([0,1])
#   en_labs.append(170)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#


Loading data/C_50keV.npy...


/tmp/ipykernel_13620/2176391138.py:27: RuntimeWarning: invalid value encountered in true_divide
  C_dat = (C_dat-np.nanmean(C_dat,axis=0))/np.nanstd(C_dat,axis=0)


Loading data/F_50keV.npy...


/tmp/ipykernel_13620/2176391138.py:42: RuntimeWarning: invalid value encountered in true_divide
  F_dat = (F_dat-np.nanmean(F_dat,axis=0))/np.nanstd(F_dat,axis=0)


Loading data/C_55keV.npy...
Loading data/F_55keV.npy...
Loading data/C_60keV.npy...
Loading data/F_60keV.npy...
Loading data/C_65keV.npy...
Loading data/F_65keV.npy...
Loading data/C_70keV.npy...
Loading data/F_70keV.npy...
Loading data/C_75keV.npy...
Loading data/F_75keV.npy...
Loading data/C_80keV.npy...
Loading data/F_80keV.npy...
Loading data/C_85keV.npy...
Loading data/F_85keV.npy...
Loading data/C_90keV.npy...
Loading data/F_90keV.npy...
Loading data/C_95keV.npy...
Loading data/F_95keV.npy...
Loading data/C_100keV.npy...
Loading data/F_100keV.npy...
Loading data/C_105keV.npy...
Loading data/F_105keV.npy...
Loading data/C_110keV.npy...
Loading data/F_110keV.npy...
Loading data/C_115keV.npy...
Loading data/F_115keV.npy...
Loading data/C_120keV.npy...
Loading data/F_120keV.npy...
Loading data/C_125keV.npy...
Loading data/F_125keV.npy...
Loading data/C_130keV.npy...
Loading data/F_130keV.npy...
Loading data/C_135keV.npy...
Loading data/F_135keV.npy...
Loading data/C_140keV.npy...
Loa

In [14]:

X_train, X_test, y_train, y_test = train_test_split(im_dat, el_labs, 
                                                    test_size=0.2, random_state=42)


In [15]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# Training squeezing #
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

xtrain_tensor = torch.from_numpy(np.array(X_train)) #converts numpy array to torch tensor
xtrain_sqz = xtrain_tensor.unsqueeze(1) #adds new dimension along axis 1
print(xtrain_sqz.shape)


ytrain_tensor = torch.from_numpy(np.array(y_train))
ytrain_sqz = ytrain_tensor.unsqueeze(1)
print(ytrain_sqz.shape)

torch.Size([240000, 1, 97, 97])
torch.Size([240000, 1])


In [16]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# Testing squeezing #
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

xtest_tensor = torch.from_numpy(np.array(X_test)) #converts numpy array to torch tensor
xtest_sqz = xtest_tensor.unsqueeze(1) #adds new dimension along axis 1
print(xtest_sqz.shape)

ytest_tensor = torch.from_numpy(np.array(y_test))
ytest_sqz = ytest_tensor.unsqueeze(1)
print(ytest_sqz.shape)

torch.Size([60000, 1, 97, 97])
torch.Size([60000, 1])


In [17]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# Custom dataset formatting #
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

train_data = cd.CustomDataset(xtrain_sqz.float(), ytrain_sqz.float())
test_data = cd.CustomDataset(xtest_sqz.float(), ytest_sqz.float())


#defining dataloader class
train_loader = tud.DataLoader(dataset=train_data, batch_size=50, shuffle=True)
train_loader_iter = iter(train_loader)

#same as above but for test data
test_loader = tud.DataLoader(dataset=test_data, batch_size=50, shuffle=True)
test_loader_iter = iter(test_loader)

In [18]:
device = torch.device('cuda')


model = Net.Net3_mod(2*len(energies)).to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5) #stochastic gradient descent used as optimiser
optimizer = optim.Adam(model.parameters(), lr=1e-4)

initial_model = model
# print(model)
numel_list = [p.numel() for p in model.parameters()]
# print(sum(numel_list), numel_list)


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#


num_epochs = 1 #setting number of epochs for CNN

#running loop training and testing over previously specified number of epochs
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    act,pred,pred_labs = test(model, device, test_loader)

Train Epoch: 1 [0/240000 (0.0%)]	Loss: 4.09567
Train Epoch: 1 [2500/240000 (1.0%)]	Loss: 3.74185
Train Epoch: 1 [5000/240000 (2.0%)]	Loss: 3.56949
Train Epoch: 1 [7500/240000 (3.0%)]	Loss: 3.48555
Train Epoch: 1 [10000/240000 (4.0%)]	Loss: 3.64515
Train Epoch: 1 [12500/240000 (5.0%)]	Loss: 3.45351
Train Epoch: 1 [15000/240000 (6.0%)]	Loss: 3.50127
Train Epoch: 1 [17500/240000 (7.0%)]	Loss: 3.45611
Train Epoch: 1 [20000/240000 (8.0%)]	Loss: 3.53782
Train Epoch: 1 [22500/240000 (9.0%)]	Loss: 3.43743
Train Epoch: 1 [25000/240000 (10.0%)]	Loss: 3.40285
Train Epoch: 1 [27500/240000 (11.0%)]	Loss: 3.60256
Train Epoch: 1 [30000/240000 (12.0%)]	Loss: 3.45678
Train Epoch: 1 [32500/240000 (14.0%)]	Loss: 3.43467
Train Epoch: 1 [35000/240000 (15.0%)]	Loss: 3.43161
Train Epoch: 1 [37500/240000 (16.0%)]	Loss: 3.46729
Train Epoch: 1 [40000/240000 (17.0%)]	Loss: 3.39112
Train Epoch: 1 [42500/240000 (18.0%)]	Loss: 3.42448
Train Epoch: 1 [45000/240000 (19.0%)]	Loss: 3.39731
Train Epoch: 1 [47500/240000 

In [ ]:
"""
plt.figure(figsize=(22,10))
i = 28
E_spec_labels = [f"C {i} keV" for i in energies]+[f"F {i} keV" for i in energies]
E_labels = np.array(list(energies) + list(energies))
plt.bar(
  E_spec_labels,
  pred[-1][i],
  edgecolor='k',
  lw=2
)
plt.xticks(rotation="vertical")
plt.ylabel("Weight")
tot_prob = np.nansum(pred[-1][i])
C_prob = np.nansum(pred[-1][i,:int(len(energies))])
F_prob = np.nansum(pred[-1][i,int(len(energies)):])
C_E = np.nansum(np.arange(int(len(energies)))*pred[-1][i,:int(len(energies))]/C_prob)
F_E = np.nansum(np.arange(int(len(energies)),2*int(len(energies)))*pred[-1][i,int(len(energies)):]/F_prob)
pred_elem = "C" if C_prob > F_prob else "F"
pred_E = C_E if C_prob > F_prob else F_E
act_elem  = "C" if act[-1][i] < int(len(energies)) else "F"

final_energy = np.round(E_labels[int(pred_E)] + 5 * (pred_E % int(pred_E)),1)

plt.title(f"Actual: {E_spec_labels[int(act[-1][i])]}\nPredicted: {pred_elem} {final_energy} keV",fontsize=20)

# plt.yscale('log')
"""